# Generative Adversarial Networks (GANs)

## Week 3, Day 1 - Deep Dive into GANs

### What You'll Learn:
1. **GAN Fundamentals** - The revolutionary idea behind GANs
2. **Generator & Discriminator** - The two-player game
3. **Training Dynamics** - How GANs learn
4. **DCGAN Architecture** - Deep Convolutional GANs
5. **🎯 Live Project**: Fashion Image Generation with DCGAN

---

## 1. What are GANs?

**Generative Adversarial Networks (GANs)** were introduced by Ian Goodfellow in 2014 and are considered one of the most exciting ideas in AI.

> *"The most interesting idea in the last 10 years in ML"* - Yann LeCun, 2016

### The Core Idea: A Two-Player Game

Imagine a game between two players:

🎨 **Generator (G)**: A forger trying to create fake art
🔍 **Discriminator (D)**: An art detective trying to catch fakes

```
                    ┌─────────────────────────────────────┐
                    │          THE GAN GAME               │
                    └─────────────────────────────────────┘
                                    
    Random Noise              Generated Image              Real/Fake?
    ┌─────────┐              ┌─────────────┐              ┌─────────┐
    │ z ~ N(0,1)│ ────────▶  │  GENERATOR  │ ────────▶   │         │
    └─────────┘              │     (G)     │              │         │
                             └─────────────┘              │ DISCRIM │ ──▶ 0 or 1
                                                          │  (D)    │
    Real Images                                           │         │
    ┌─────────┐                              ────────▶   │         │
    │ Dataset │                                           └─────────┘
    └─────────┘
```

### The Adversarial Process:

1. **Generator** creates fake images from random noise
2. **Discriminator** tries to distinguish real from fake
3. Both improve through competition
4. Eventually, Generator creates images so realistic that Discriminator can't tell!

## 2. Mathematical Foundation

### The Minimax Game

GANs optimize this objective function:

$$\min_G \max_D V(D, G) = \mathbb{E}_{x \sim p_{data}(x)}[\log D(x)] + \mathbb{E}_{z \sim p_z(z)}[\log(1 - D(G(z)))]$$

**Breaking it down:**

| Term | Meaning |
|------|----------|
| $D(x)$ | Discriminator's probability that x is real |
| $G(z)$ | Generator's output from noise z |
| $\log D(x)$ | D wants this HIGH (correctly identify real) |
| $\log(1 - D(G(z)))$ | D wants this HIGH (correctly identify fakes) |
| | G wants this LOW (fool the discriminator) |

### Training Objectives:

**Discriminator's Goal**: Maximize ability to distinguish real from fake
```
D wants: D(real) → 1 and D(G(z)) → 0
```

**Generator's Goal**: Minimize discriminator's ability to detect fakes
```
G wants: D(G(z)) → 1 (fool D into thinking fakes are real)
```

---
## 3. Setup & Installation

Let's set up our environment for building GANs!

In [ ]:
# Install dependencies (uncomment for Colab)
# !pip install tensorflow matplotlib imageio tqdm

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import time
import os

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

---
## 4. Building a Simple GAN from Scratch

Let's first build a basic GAN to understand the mechanics before moving to DCGAN.

In [ ]:
# Simple Generator Network
def build_simple_generator(latent_dim):
    """Generator: Maps random noise to image space."""
    model = keras.Sequential([
        layers.Dense(256, input_dim=latent_dim),
        layers.LeakyReLU(0.2),
        layers.BatchNormalization(),
        
        layers.Dense(512),
        layers.LeakyReLU(0.2),
        layers.BatchNormalization(),
        
        layers.Dense(1024),
        layers.LeakyReLU(0.2),
        layers.BatchNormalization(),
        
        layers.Dense(28 * 28 * 1, activation='tanh'),
        layers.Reshape((28, 28, 1))
    ], name='generator')
    return model

# Simple Discriminator Network
def build_simple_discriminator(img_shape):
    """Discriminator: Classifies images as real or fake."""
    model = keras.Sequential([
        layers.Flatten(input_shape=img_shape),
        
        layers.Dense(512),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.3),
        
        layers.Dense(256),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.3),
        
        layers.Dense(1, activation='sigmoid')
    ], name='discriminator')
    return model

# Test the architectures
latent_dim = 100
img_shape = (28, 28, 1)

gen = build_simple_generator(latent_dim)
disc = build_simple_discriminator(img_shape)

print("Generator Summary:")
gen.summary()
print("\nDiscriminator Summary:")
disc.summary()

---
## 5. DCGAN - Deep Convolutional GAN

DCGAN (2015) introduced architectural guidelines that made training stable:

### DCGAN Guidelines:

| Guideline | Generator | Discriminator |
|-----------|-----------|---------------|
| Convolutions | Use Transposed Conv | Use Strided Conv |
| Batch Norm | Yes (except output) | Yes (except input) |
| Activation | ReLU | LeakyReLU |
| Output Activation | Tanh | Sigmoid |
| Pooling | No Max Pooling | No Max Pooling |

```
DCGAN Generator Architecture:
┌───────────┐    ┌───────────┐    ┌───────────┐    ┌───────────┐    ┌───────────┐
│  Noise z  │ ─▶ │ Dense +   │ ─▶ │ ConvT 256 │ ─▶ │ ConvT 128 │ ─▶ │ ConvT 64  │ ─▶ Image
│  (100,)   │    │ Reshape   │    │   7×7     │    │   14×14   │    │   28×28   │
└───────────┘    └───────────┘    └───────────┘    └───────────┘    └───────────┘
```

In [ ]:
# DCGAN Generator
def build_dcgan_generator(latent_dim=100):
    model = keras.Sequential(name='dcgan_generator')
    
    # Foundation for 7x7 image
    model.add(layers.Dense(7 * 7 * 256, use_bias=False, input_shape=(latent_dim,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Reshape((7, 7, 256)))
    
    # Upsample to 7x7x128
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    # Upsample to 14x14x64
    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    # Upsample to 28x28x1
    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    
    return model

# DCGAN Discriminator
def build_dcgan_discriminator():
    model = keras.Sequential(name='dcgan_discriminator')
    
    # 28x28x1 -> 14x14x64
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    # 14x14x64 -> 7x7x128
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    # Flatten and classify
    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    
    return model

# Build models
generator = build_dcgan_generator()
discriminator = build_dcgan_discriminator()

print("DCGAN Generator:")
generator.summary()
print("\nDCGAN Discriminator:")
discriminator.summary()

In [ ]:
# Test the untrained generator
noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

plt.figure(figsize=(4, 4))
plt.imshow(generated_image[0, :, :, 0], cmap='gray')
plt.title('Untrained Generator Output (Random Noise)')
plt.axis('off')
plt.show()

# Test discriminator
decision = discriminator(generated_image)
print(f"Discriminator output: {decision.numpy()[0][0]:.4f}")
print("(Before training, this is random)")

---
## 6. 🎯 Live Project: Fashion Image Generation

We'll train a DCGAN to generate realistic fashion items using the **Fashion-MNIST** dataset!

### Why Fashion-MNIST?
- **Real-world relevance**: Fashion industry uses GANs for design
- **Challenging**: More complex than digits
- **Fast training**: 28x28 images train quickly
- **Clear quality metrics**: Easy to evaluate output

In [ ]:
# Load Fashion-MNIST dataset
(train_images, train_labels), (_, _) = keras.datasets.fashion_mnist.load_data()

# Fashion-MNIST class labels
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print(f"Dataset shape: {train_images.shape}")
print(f"Number of training images: {len(train_images):,}")

# Visualize some samples
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    ax.imshow(train_images[i], cmap='gray')
    ax.set_title(class_names[train_labels[i]])
    ax.axis('off')
plt.suptitle('Fashion-MNIST Samples', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Preprocess the data

# Normalize to [-1, 1] (for tanh activation)
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5

print(f"Min value: {train_images.min()}, Max value: {train_images.max()}")

# Create tf.data.Dataset for efficient batching
BUFFER_SIZE = 60000
BATCH_SIZE = 256

train_dataset = tf.data.Dataset.from_tensor_slices(train_images)
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

print(f"Number of batches: {len(train_dataset)}")

In [ ]:
# Define loss function and optimizers

cross_entropy = keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    """Discriminator wants to correctly classify real and fake."""
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)  # Real = 1
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)  # Fake = 0
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    """Generator wants discriminator to think fakes are real."""
    return cross_entropy(tf.ones_like(fake_output), fake_output)  # Wants D(G(z)) = 1

# Optimizers with recommended learning rates
generator_optimizer = keras.optimizers.Adam(1e-4)
discriminator_optimizer = keras.optimizers.Adam(1e-4)

print("✅ Loss functions and optimizers defined!")

In [ ]:
# Training configuration
LATENT_DIM = 100
EPOCHS = 50  # Increase for better results
num_examples_to_generate = 16

# Seed for consistent visualization
seed = tf.random.normal([num_examples_to_generate, LATENT_DIM])

@tf.function
def train_step(images):
    """Single training step for both G and D."""
    noise = tf.random.normal([BATCH_SIZE, LATENT_DIM])
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
    
    # Compute and apply gradients
    gen_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
    disc_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    generator_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(disc_gradients, discriminator.trainable_variables))
    
    return gen_loss, disc_loss

In [ ]:
def generate_and_save_images(model, epoch, test_input):
    """Generate and display images during training."""
    predictions = model(test_input, training=False)
    
    fig = plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')
    
    plt.suptitle(f'Epoch {epoch}', fontsize=12)
    plt.tight_layout()
    plt.show()

# Test visualization function
print("Initial random output:")
generate_and_save_images(generator, 0, seed)

In [ ]:
# Training loop
def train(dataset, epochs):
    gen_losses = []
    disc_losses = []
    
    for epoch in range(epochs):
        start = time.time()
        
        epoch_gen_loss = []
        epoch_disc_loss = []
        
        for image_batch in dataset:
            g_loss, d_loss = train_step(image_batch)
            epoch_gen_loss.append(g_loss)
            epoch_disc_loss.append(d_loss)
        
        # Track average losses
        gen_losses.append(np.mean(epoch_gen_loss))
        disc_losses.append(np.mean(epoch_disc_loss))
        
        # Display progress every 10 epochs
        if (epoch + 1) % 10 == 0:
            display.clear_output(wait=True)
            generate_and_save_images(generator, epoch + 1, seed)
            print(f'Epoch {epoch+1}/{epochs}')
            print(f'  Generator Loss: {gen_losses[-1]:.4f}')
            print(f'  Discriminator Loss: {disc_losses[-1]:.4f}')
            print(f'  Time: {time.time()-start:.2f}s')
    
    # Final generation
    display.clear_output(wait=True)
    generate_and_save_images(generator, epochs, seed)
    
    return gen_losses, disc_losses

print("🚀 Starting training...")
print(f"Training for {EPOCHS} epochs with batch size {BATCH_SIZE}")
print("This may take 10-20 minutes depending on your hardware.\n")

In [ ]:
# Run training!
gen_losses, disc_losses = train(train_dataset, EPOCHS)

In [ ]:
# Plot training progress
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(gen_losses, label='Generator', linewidth=2)
plt.plot(disc_losses, label='Discriminator', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('GAN Training Losses')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(gen_losses, label='Generator', linewidth=2, alpha=0.7)
plt.xlabel('Epoch')
plt.ylabel('Generator Loss')
plt.title('Generator Loss Over Time')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 7. Generate New Fashion Items!

Now let's use our trained generator to create new fashion designs!

In [ ]:
# Generate a grid of fashion items
def generate_fashion_grid(generator, n_rows=4, n_cols=8):
    """Generate a grid of fashion items."""
    n_samples = n_rows * n_cols
    noise = tf.random.normal([n_samples, LATENT_DIM])
    generated = generator(noise, training=False)
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 8))
    for i, ax in enumerate(axes.flat):
        img = generated[i, :, :, 0] * 127.5 + 127.5
        ax.imshow(img, cmap='gray')
        ax.axis('off')
    
    plt.suptitle('🎨 AI-Generated Fashion Items', fontsize=16)
    plt.tight_layout()
    plt.show()

generate_fashion_grid(generator)

In [ ]:
# Latent Space Interpolation - Watch fashion morph!
def interpolate_latent_space(generator, n_steps=10):
    """Interpolate between two random points in latent space."""
    # Two random latent vectors
    z1 = tf.random.normal([1, LATENT_DIM])
    z2 = tf.random.normal([1, LATENT_DIM])
    
    # Linear interpolation
    ratios = np.linspace(0, 1, n_steps)
    
    fig, axes = plt.subplots(1, n_steps, figsize=(20, 3))
    
    for i, ratio in enumerate(ratios):
        z = z1 * (1 - ratio) + z2 * ratio
        img = generator(z, training=False)
        axes[i].imshow(img[0, :, :, 0] * 127.5 + 127.5, cmap='gray')
        axes[i].axis('off')
        axes[i].set_title(f'{ratio:.1f}')
    
    plt.suptitle('Latent Space Interpolation: Morphing Fashion', fontsize=14)
    plt.tight_layout()
    plt.show()

print("Watch one fashion item morph into another!")
interpolate_latent_space(generator)

---
## 8. Real vs Generated Comparison

In [ ]:
# Compare real and generated images side by side
fig, axes = plt.subplots(2, 8, figsize=(16, 4))

# Real images (top row)
for i in range(8):
    axes[0, i].imshow(train_images[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
    axes[0, i].axis('off')
    if i == 0:
        axes[0, i].set_title('Real Images', fontsize=12, loc='left')

# Generated images (bottom row)
noise = tf.random.normal([8, LATENT_DIM])
generated = generator(noise, training=False)
for i in range(8):
    axes[1, i].imshow(generated[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
    axes[1, i].axis('off')
    if i == 0:
        axes[1, i].set_title('Generated Images', fontsize=12, loc='left')

plt.suptitle('Real vs AI-Generated Fashion', fontsize=14)
plt.tight_layout()
plt.show()

---
## 9. Types of GANs

| GAN Type | Purpose | Example Use Case |
|----------|---------|------------------|
| **DCGAN** | Image generation with CNNs | Fashion, faces |
| **Conditional GAN** | Generate specific classes | Generate specific fashion items |
| **CycleGAN** | Unpaired image translation | Horse ↔ Zebra, Photo ↔ Painting |
| **StyleGAN** | High-quality face generation | ThisPersonDoesNotExist.com |
| **Pix2Pix** | Paired image translation | Sketch → Photo |
| **SRGAN** | Super-resolution | Enhance low-res images |

### Famous GAN Applications:

1. **DeepFakes** - Face swapping in videos
2. **DALL-E/Midjourney** - Text to image (uses GAN concepts)
3. **NVIDIA GauGAN** - Landscape painting tool
4. **AI Art Generation** - Creating artwork

---
## 10. GAN Training Challenges & Solutions

### Common Problems:

| Problem | Description | Solution |
|---------|-------------|----------|
| **Mode Collapse** | Generator produces limited variety | Use minibatch discrimination, unrolled GANs |
| **Training Instability** | Loss oscillates wildly | Use WGAN, spectral normalization |
| **Vanishing Gradients** | D becomes too strong | Use feature matching, label smoothing |
| **Non-convergence** | Training never stabilizes | Use progressive growing, careful hyperparams |

In [ ]:
# Demonstration: Label Smoothing for stable training
def discriminator_loss_with_smoothing(real_output, fake_output, smoothing=0.1):
    """Use label smoothing: real=0.9 instead of 1.0"""
    real_labels = tf.ones_like(real_output) * (1 - smoothing)  # 0.9 instead of 1
    fake_labels = tf.zeros_like(fake_output)
    
    real_loss = cross_entropy(real_labels, real_output)
    fake_loss = cross_entropy(fake_labels, fake_output)
    return real_loss + fake_loss

print("💡 Label smoothing prevents discriminator from becoming overconfident!")

---
## 11. Summary & Key Takeaways

### What We Learned:

| Concept | Description |
|---------|-------------|
| **GANs** | Two networks in adversarial training |
| **Generator** | Creates fake data from random noise |
| **Discriminator** | Distinguishes real from fake |
| **DCGAN** | Convolutional GANs for images |
| **Latent Space** | Compressed representation of data |
| **Mode Collapse** | When G produces limited variety |

### Training Tips:
1. Use **BatchNorm** in Generator
2. Use **LeakyReLU** in Discriminator
3. Apply **label smoothing**
4. Balance G and D training
5. Monitor both losses

### Next Steps:
- Try **Conditional GANs** for class-specific generation
- Explore **StyleGAN** for high-quality faces
- Learn about **CycleGAN** for image translation

---
## 12. 📝 Practice Exercises

### Exercise 1: Train on MNIST Digits
Modify the code to train on MNIST digits instead of Fashion-MNIST.

### Exercise 2: Experiment with Architecture
Try adding more layers or changing the number of filters. How does it affect quality?

### Exercise 3: Conditional Generation (Advanced)
Modify the GAN to generate specific fashion categories by conditioning on class labels.

### Exercise 4: Training Duration
Train for 100+ epochs. Document the quality improvement over time.

In [ ]:
# YOUR EXERCISE SOLUTIONS HERE

# Exercise 1: MNIST Training
# Hint: Just change keras.datasets.fashion_mnist to keras.datasets.mnist

# Exercise 2: Architecture changes
# Hint: Try adding layers.Conv2DTranspose(256, ...) layer

# Exercise 3: Conditional GAN
# Hint: Concatenate class labels with noise input to generator


---
## 13. 📚 Additional Resources

### Papers:
- [Original GAN Paper](https://arxiv.org/abs/1406.2661) - Goodfellow et al., 2014
- [DCGAN Paper](https://arxiv.org/abs/1511.06434) - Radford et al., 2015
- [StyleGAN](https://arxiv.org/abs/1812.04948) - Karras et al., 2018

### Interactive Tools:
- [GAN Lab](https://poloclub.github.io/ganlab/) - Visualize GAN training in browser
- [ThisPersonDoesNotExist.com](https://thispersondoesnotexist.com) - StyleGAN faces

### Tutorials:
- [TensorFlow DCGAN Tutorial](https://www.tensorflow.org/tutorials/generative/dcgan)
- [PyTorch GAN Tutorial](https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html)

---

**🎉 Congratulations! You've built and trained a GAN!**

*Next: Transformers & Attention Mechanisms*